In [49]:
import itertools
import torch
import pickle
import numpy as np
import ZH_Nakamura
from Class_R import R
torch.set_default_dtype(torch.float64)

In [50]:
def f_loss(r,features,base_points):
        loss=0
        for i_base_point, base_point in enumerate(base_points):
            fhat  = torch.tensor(1./(1. + r.predict_r_hat(features,base_point)), requires_grad=True)
            loss += (fhat**2 + (1-fhat)**2).sum()
        return loss

In [57]:
class R:
    def __init__(self,nfeatures,coefficient_names):
        self.nfeatures         = nfeatures
        self.coefficient_names = coefficient_names
        self.combination_list=list(itertools.chain.from_iterable(itertools.combinations_with_replacement(self.coefficient_names, i) for i in np.arange(0,3)))
        self.n_hat = {combination:self.make_NN() for combination in self.combination_list}
        
    def make_NN(self, hidden_layers  = [32, 32, 32, 32]):
        model_nn = [torch.nn.BatchNorm1d(self.nfeatures), torch.nn.ReLU(), torch.nn.Linear(self.nfeatures, hidden_layers[0])]
        for i_layer, layer in enumerate(hidden_layers):
            model_nn.append(torch.nn.Linear(hidden_layers[i_layer], hidden_layers[i_layer+1] if i_layer+1<len(hidden_layers) else 1))
            if i_layer+1<len(hidden_layers):
                model_nn.append( torch.nn.ReLU() )
        return torch.nn.Sequential(*model_nn)

    def evaluate_NN(self, features):
        '''Evaluate Neural Network: The zeroth dimension of features is the number of data points and and the first dimension
        is the number of features(variables)
        '''
        noutputs=len(self.combination_list)
        ndatapoints=features.shape[0]
        output=np.zeros((noutputs,ndatapoints))
        #print("Output Shape:")
        #print(output.shape)
        #print("Features Shape:")
        #print(features.shape)
            
        for i in range(noutputs):
            x=self.n_hat[self.combination_list[i]](features).detach().numpy()
            if i==0:
                output[i,:]=1
            else:
                output[i,:]=x.flatten()            
            
       # print("Output is:")
        #print(output)
        return output
        
    
    def predict_r_hat(self, features, theta):
        '''Evaluate positive xsec ratio for given theta and 
        '''
        ndatapoints=features.shape[0]
        output_NN = self.evaluate_NN(features)
        #print("Output_NN shape")
        #print(output_NN.shape)
        n_terms=len(self.coefficient_names)
        #print('N terms: '+str(n_terms))
        row,column=np.triu_indices(n_terms)
        Omega=np.zeros((n_terms+1,n_terms+1,ndatapoints))
        #print("Length of Row")
        #print(len(row))
        #print("Shape of Omega")
        #print(Omega.shape)
        for i in range(len(row)):
            Omega[row[i]][column[i]][:]=output_NN[i,:]
        Omega_swapped=np.swapaxes(Omega,1,2)
        Omega_swapped=np.swapaxes(Omega_swapped,0,1)
        
        #print("Shape of Omega swapped")
        #print(Omega_swapped.shape)
        #print("Shape of theta")
        #print(theta.shape)
        
        out=np.matmul(Omega_swapped, theta)        
        #print("Out Shape")
        #print(out.shape)
        return torch.tensor(np.linalg.norm(out, 2, 1)**2, requires_grad=True)
    
    def save(self,fileName):
        outfile = open(fileName,'wb')
        pickle.dump(self, outfile)
        outfile.close()
        
    @classmethod
    def load(self, fileName):
        infile = open(fileName,'rb')
        print(fileName)
        new_dict = pickle.load(infile)
        infile.close()
        return new_dict

In [58]:
n_features=6
coefficients=['cHW', 'cHWtil', 'cHQ3']
base_points = [np.array([1, value1, value2, value3]) for value1 in [-1.5, -.8, .2, 0., .2, .8, 1.5]  for value2 in [-1.5, -.8, .2, 0, .2, .8, 1.5] for value3 in [-1.5, -.8, .2, 0, .2, .8, 1.5]]
n_datapoints=2
r = R(n_features, coefficients)


In [59]:
plot_directory="v1_with_Biasing"

nEvents=100
learning_rate = 1e-3
device        = 'cuda' if torch.cuda.is_available() else 'cpu'
n_epoch       = 10
plot_every    = 100

In [60]:
# training data
import ZH_Nakamura 
ZH_Nakamura.feature_names = ZH_Nakamura.feature_names[0:6] # restrict features
features   = ZH_Nakamura.getEvents(nEvents)[:,0:6]
feature_names  = ZH_Nakamura.feature_names
plot_options   = ZH_Nakamura.plot_options
plot_vars      = ZH_Nakamura.feature_names

mask       = (features[:,feature_names.index('pT')]<900) & (features[:,feature_names.index('sqrt_s_hat')]<1800) 
features = features[mask]

n_features = len(features[0]) 
weights    = ZH_Nakamura.getWeights(features, ZH_Nakamura.make_eft())


pT=features[:,feature_names.index('pT')]
features=torch.from_numpy(features)
    
#for key,value in weights.items():
 #   value*=bias_factor
  #  weights[key]=value

Requested 100 events. Simulated 100 events and 100 survive pT_min cut of 0.


In [77]:
all_params=[]
for comb in r.combination_list:
    all_params+=r.n_hat[comb].parameters()

In [78]:
#optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(list(all_params), lr=learning_rate)
losses = []

In [85]:
for epoch in range(n_epoch):
    # Forward pass: compute predicted y by passing x to the model.
    predictions = {combination:r.n_hat[combination](features).squeeze() for combination in r.combination_list}

    # Compute and print loss.
    
    #loss = torch.tensor(f_loss(r,features,base_points), requires_grad=True)
    loss = f_loss(r,features,base_points)
    loss.requires_grad_(True)
    print("Loss:")
    print(loss)
    losses.append(loss.item())
    #if epoch % 100 == 99:
    #    print("epoch", epoch, "loss",  loss.item())

    loss.backward()
    
    # update weights
    optimizer.step()

    # zero the gradients after updating
    optimizer.zero_grad()
        


Loss:
tensor(10280.3270, grad_fn=<AddBackward0>)
Loss:
tensor(10280.3270, grad_fn=<AddBackward0>)
Loss:
tensor(10280.3270, grad_fn=<AddBackward0>)
Loss:
tensor(10280.3270, grad_fn=<AddBackward0>)
Loss:
tensor(10280.3270, grad_fn=<AddBackward0>)
Loss:
tensor(10280.3270, grad_fn=<AddBackward0>)
Loss:
tensor(10280.3270, grad_fn=<AddBackward0>)
Loss:
tensor(10280.3270, grad_fn=<AddBackward0>)
Loss:
tensor(10280.3270, grad_fn=<AddBackward0>)
Loss:
tensor(10280.3270, grad_fn=<AddBackward0>)
